In [ ]:
from neural_network.neural_network import FeedForward
from forward_process.generate_noised_data import GenerateNoisedData
from neural_network.preprocessing import Preprocessing
from neural_network.training_nn import Train
from initial_distributions.two_deltas import GenerateTwoDeltas
from save_plot.save_files import LoadCSV

import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler

device = torch.device(f'cuda:{torch.cuda.current_device()}') if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)


In [ ]:
def TrainModel(timesteps, ndata, initial_distribution):

    model = FeedForward(input_size=2,output_size=1,n_hidden_layers=2,depht=200).to(device)

    features, noise = LoadCSV()
    
    scaler = StandardScaler()

    features = scaler.fit_transform(features)

    train_dl, valid_dl, test_feature, test_target = Preprocessing(features, noise)

    loss_hist_train, loss_hist_valid = Train(learning_rate=0.01, model=model, num_epochs=30,
                                           train_dl=train_dl, valid_dl=valid_dl
                                           )

    pred = model(test_feature)

    loss_fn = nn.MSELoss()

    test_loss = loss_fn(pred, test_target).item()

    print(f'test error:  {test_loss}')

    torch.save(model.state_dict(), 'model/FeedForward.pth')

    return loss_hist_train, loss_hist_valid, scaler

In [ ]:
ndata = 200
train_times = 1000

initial_distribution = GenerateTwoDeltas(ndata)
GenerateNoisedData(train_times, ndata, initial_distribution)

Generating two deltas distribution with 200 data points...


In [ ]:
TrainModel(train_times, ndata, initial_distribution)